In [5]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import inference_detector, init_detector, show_result_pyplot
from mmcv import Config
import copy
import os.path as osp
import os
import mmcv
import numpy as np
import time
import cv2
import matplotlib.pyplot as plt
import matplotlib
from pylab import *
from matplotlib import cm

os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

def sigmoid(x):
    # TODO: Implement sigmoid function
    return 1/(1 + np.exp(-x))

config_xiaohei='./configs/faster_rcnn/xiaohei_faster_rcnn_r50_fpn_1x_chenzhou.py'
config_faster='./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_chenzhou.py'
config_mask='./configs/mask_rcnn/mask_rcnn_r50_fpn_1x_chenzhou.py'
config_ms='./configs/ms_rcnn/ms_rcnn_r50_fpn_1x_chenzhou.py'

work_xiaohei='logs/xiaohei_faster_rcnn_r50_fpn_1x_chenzhou_202011141101/'
work_faster='logs/faster_rcnn_r50_fpn_1x_chenzhou_202011260631/'
work_mask='logs/mask_rcnn_r50_fpn_1x_chenzhou_202011250925/'
work_ms='logs/ms_rcnn_r50_fpn_1x_chenzhou_202011251546/'

configDict = [
                ['xiaohei', config_xiaohei, work_xiaohei]
                ,['faster', config_faster, work_faster]
                ,['mask', config_mask, work_mask]
                ,['ms', config_ms, work_ms]
             ]

f = open('logs/processInference-Proposal.log', 'w+')

for config_type, config_file, work_dir in configDict:
    cfg = Config.fromfile(config_file)
    cfg.work_dir = work_dir
    cfg.seed = 0
    cfg.gpu_ids = range(0,2)
    cfg.total_epochs = 50
    cfg.log_config.interval = 1000

    checkpoint = cfg.work_dir + 'latest.pth'

    model = init_detector(cfg, checkpoint, device='cuda:0')
    datasets = [build_dataset(cfg.data.train)]
    model.CLASSES = datasets[0].CLASSES
    # validations
    result_path = cfg.work_dir + '/tests/'
    text=config_type + ':'
    # os.linesep代表当前操作系统上的换行符

    path = '/root/workspace/Thyroid_Solid_Nodule/data/preprocess/chenzhou_aug/tests/'
    for root, dirs, files in os.walk(os.path.abspath(path)):
        for file in files:
            img = mmcv.imread(path + file)
            if 'mask_head' in cfg.model.roi_head:
                result, a, cls_score, bbox_pred = inference_detector(model, img)
            else:
                result, cls_score, bbox_pred = inference_detector(model, img)
            for index, feat in enumerate(cls_score.data.cpu().numpy()):
#                 print(feat)
                n = sigmoid(float(feat[0]))
                b = sigmoid(float(feat[-1]))
#                 print(text, "%.8f" % (n/sum([n, b])), "%.8f" % (b/sum([n, b])))
                f.write(text + "%.8f" % (n/sum([n, b])) + os.linesep)

    path = '/root/workspace/Thyroid_Solid_Nodule/data/preprocess/chenzhou_aug/validations/'
    for root, dirs, files in os.walk(os.path.abspath(path)):
        for file in files:
            img = mmcv.imread(path + file)
            if 'mask_head' in cfg.model.roi_head:
                result, a, cls_score, bbox_pred = inference_detector(model, img)
            else:
                result, cls_score, bbox_pred = inference_detector(model, img)
            for index, feat in enumerate(cls_score.data.cpu().numpy()):
#                 print(feat)
                n = sigmoid(float(feat[0]))
                b = sigmoid(float(feat[-1]))
#                 print(text, "%.8f" % (n/sum([n, b])), "%.8f" % (b/sum([n, b])))
                f.write(text + "%.8f" % (n/sum([n, b])) + os.linesep)

    path = '/root/workspace/Thyroid_Solid_Nodule/data/preprocess/chenzhou/image/'
    for root, dirs, files in os.walk(os.path.abspath(path)):
        for file in files:
            img = mmcv.imread(path + file)
            if 'mask_head' in cfg.model.roi_head:
                result, a, cls_score, bbox_pred = inference_detector(model, img)
            else:
                result, cls_score, bbox_pred = inference_detector(model, img)
            for index, feat in enumerate(cls_score.data.cpu().numpy()):
#                 print(feat)
                n = sigmoid(float(feat[0]))
                b = sigmoid(float(feat[-1]))
#                 print(text, "%.8f" % (n/sum([n, b])), "%.8f" % (b/sum([n, b])))
                f.write(text + "%.8f" % (n/sum([n, b])) + os.linesep)
f.close()

RuntimeError: Attempting to deserialize object on CUDA device 1 but torch.cuda.device_count() is 1. Please use torch.load with map_location to map your storages to an existing device.